In [ ]:
import pathlib
import sys
import shutil
import tempfile
import os
import subprocess
import csv
import biom
import numpy as np
import pandas as pd
from qiime2 import Artifact, Metadata
from qiime2.plugins import cutadapt 
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.dada2.methods import denoise_single
from qiime2.plugins.feature_table.visualizers import summarize as summarize_table

In [ ]:
metadata = Metadata.load('../sample_data/IndexJumping/Sample_Map.txt')
demux = Artifact.load('../sample_data/IndexJumping/demultiplexed_seqs.qza')

In [ ]:
#Trimming ANML primers off the demultiplexed reads
ANML_primers = ["^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC"]

trimmed_demux = cutadapt.methods.trim_single(demux, 
                                             cores=16, 
                                             adapter = ANML_primers, 
                                             indels = True,
                                             minimum_length = 170, 
                                             discard_untrimmed = True)


In [ ]:
summarize(trimmed_demux.trimmed_sequences).visualization

In [ ]:
#DADA2
feature_table, rep_seqs, stats = denoise_single(trimmed_demux.trimmed_sequences,
                                                trunc_len = 0, 
                                                n_threads = 0)

In [ ]:
feature_table.save('../sample_data/PerSampleFiltering/feature_table.qza')
summarize_table(feature_table).visualization

In [ ]:
# Extract .biom from feature table file if necessary
def extract_all(file, dest):
    with tempfile.TemporaryDirectory() as temp:
        file.export_data(temp)
        temp_pathlib = pathlib.Path(temp)
        for file in temp_pathlib.iterdir():
            shutil.copy(file, dest)
extract_all(feature_table, 'out')